In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

df=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/ins/13/hfqtl_NelVar13.csv")
df.shape

(4763, 36)

In [ ]:
df_qtl=df[df["qtlgene_type"]=="within_gene"]
df_qtl.shape

(3155, 36)

In [ ]:
df_hf=df_qtl[df_qtl["HFgene_type"]=="within_gene"]
df_hf.shape

(3016, 36)

In [ ]:
df_nel=df_hf[df_hf["Nelgene_type"]=="within_gene"]
df_nel.shape

(2588, 36)

In [ ]:
ins_df=df_nel.drop_duplicates(subset=["Symbol", "QTL Class", "startpos.q", "endpos.q", "NelLocus", "LOV"]).reset_index()
ins_df.shape

(1602, 37)

In [ ]:
ins_dff=ins_df.groupby(["Name", "Symbol", "NelLocus", "NelProt", "startpos.q", "endpos.q", "LOV"]).agg({"QTL Class": lambda x: ",".join(set(x))}).reset_index()
ins_dff.shape

(566, 8)

In [ ]:
ins_dff.to_csv("ins_dff.csv")

In [ ]:
ins_dff["count"] = ins_dff.groupby(by=["Name", 'Symbol', 'NelLocus', "NelProt"])["Symbol"].transform("count")
ins_dff.shape

(566, 9)

In [ ]:
ins_dff['LOV >= 50'] = (ins_dff['LOV'] >= 10).astype(int)
ins_dff['Length'] = ins_dff['LOV'].where(ins_dff['LOV'] >= 10, None)

In [ ]:
result_df = ins_dff.groupby(["Name", 'Symbol', 'NelLocus', "NelProt"]).agg({
    'QTL Class': lambda x: ', '.join(set(x)),
    'count': 'first',
    'LOV >= 50': 'sum',  # Change to sum to count occurrences of LOV >= 50
    'Length': lambda x: ', '.join(map(str, x.dropna().astype(int))) if not x.dropna().empty else None
}).reset_index()

In [ ]:
result_df.to_csv("result_df.csv", index=False)

In [ ]:
result_df.shape

(153, 9)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame with the given data
# Assuming ins_df is your DataFrame with the given data

# Step 1: Add a count column
ins_df["count"] = ins_df.groupby(by=["Name", 'Symbol', 'NelLocus', "NelProt"])["Symbol"].transform("count")

# Step 2: Check Count >= 50
ins_df['count >= 50'] = (ins_df['count'] >= 50).astype(int)

# Step 3: Add LOV >= 50 and Length columns
ins_df['LOV >= 50'] = (ins_df['LOV'] >= 10).astype(int)
ins_df['Length'] = ins_df['LOV'].where(ins_df['LOV'] >= 10, None)

# Step 4: Aggregate QTL Class and other columns
result_df = ins_df.groupby(["Name", 'Symbol', 'NelLocus', "NelProt"]).agg({
    'QTL Class': lambda x: ', '.join(set(x)),
    'count': 'first',
    'count >= 50': 'max',
    'LOV >= 50': 'sum',  # Change to sum to count occurrences of LOV >= 50
    'Length': lambda x: ', '.join(map(str, x.dropna().astype(int))) if not x.dropna().empty else None
}).reset_index()

# Display the result
print(result_df)

result_df.to_csv("result_df.csv", index=False)

                                          Name        Symbol NelLocus  \
0           3-hydroxyisobutyrate dehydrogenase        HIBADH   HIBADH   
1                 A-kinase anchoring protein 9         AKAP9    AKAP9   
2              ADAM metallopeptidase domain 22        ADAM22   ADAM22   
3                      AGBL carboxypeptidase 3         AGBL3    AGBL3   
4    ATP binding cassette subfamily B member 5         ABCB5    ABCB5   
..                                         ...           ...      ...   
155                    zinc finger protein 277        ZNF277   ZNF277   
156                zinc finger protein 44-like  LOC112446469  ZNF804B   
157                    zinc finger protein 746        ZNF746   ZNF746   
158                   zinc finger protein 804B       ZNF804B  ZNF804B   
159             zona pellucida binding protein          ZPBP     ZPBP   

                                               NelProt  \
0    PREDICTED: 3-hydroxyisobutyrate dehydrogenase,...   
1      

In [ ]:
# insertion analysis

import shutil

chr=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,"X"]
for i in chr:
  in_f1=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/ins/{i}/hfqtl_NelVar{i}.csv"
  op_loc=f"/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/{i}"
  print(f"chromosome{i}")

  df=pd.read_csv(in_f1)
  print("file:", df.shape)

  df_qtl=df[df['qtlgene_type']=="within_gene"]
  print("df_qtl:", df_qtl.shape)

  df_hf=df_qtl[df_qtl["HFgene_type"]=="within_gene"]
  print("df_hf:", df_hf.shape)

  df_nel=df_hf[df_hf["Nelgene_type"]=="within_gene"]
  print("df_nel:", df_nel.shape)

  ins_df=df_nel.drop_duplicates(subset=["Symbol", "QTL Class", "startpos.q", "endpos.q", "LOV"]).reset_index()
  print("ins_df:", ins_df.shape)

  out_f1=f"HFNelQTL_wd{i}.csv"
  ins_df.to_csv(out_f1)
  shutil.copy(out_f1, op_loc)

  ins_dff=ins_df.groupby(["Name", "Symbol", "NelLocus", "NelProt", "startpos.q", "endpos.q", "LOV"]).agg({"QTL Class": lambda x: ",".join(set(x))}).reset_index()
  print("ins_dff:",ins_dff.shape)

  out_ff1=f"HFNelQTL{i}.csv"
  ins_dff.to_csv(out_ff1)
  shutil.copy(out_ff1, op_loc)


  # Add a count column
  ins_dff["ins_count"]=ins_dff.groupby(["Name", "Symbol", "NelLocus", "NelProt"])["Symbol"].transform("count")

  # Add LOV >= 10 and Length columns
  ins_dff["ins_count_LOV10"]=(ins_dff["LOV"]>=10).astype(int)
  ins_dff["Length_LOV10"]=ins_dff["LOV"].where(ins_dff["LOV"]>=10, None)

  # Add LOV>=50 and its length columns
  ins_dff["ins_count_LOV50"]=(ins_dff["LOV"]>=50).astype(int)
  ins_dff["Length_LOV50"]=ins_dff["LOV"].where(ins_dff["LOV"]>=50, None)

  # Aggregate QTL Class and other columns
  result_dff=ins_dff.groupby(["Name", "Symbol", "NelLocus", "NelProt"]).agg({"QTL Class" : lambda x: ",".join(set(x)),
    "ins_count": "first", "ins_count_LOV10" : 'sum',
    "Length_LOV10" :lambda x: ', '.join(map(str, x.dropna().astype(int))) if not x.dropna().empty else None,
    "ins_count_LOV50" : 'sum',
    "Length_LOV50" :lambda x: ', '.join(map(str, x.dropna().astype(int))) if not x.dropna().empty else None }).reset_index()

  out_f2=f"inslov_count{i}.csv"
  result_dff.to_csv(out_f2)
  print("result_dff:", result_dff.shape)
  shutil.copy(out_f2, op_loc)








chromosome1
file: (9583, 36)
df_qtl: (5868, 36)
df_hf: (5741, 36)
df_nel: (5270, 36)
ins_df: (2867, 37)
ins_dff: (937, 8)
result_dff: (163, 10)
chromosome2
file: (6229, 36)
df_qtl: (3817, 36)
df_hf: (3590, 36)
df_nel: (3047, 36)
ins_df: (1866, 37)
ins_dff: (640, 8)
result_dff: (164, 10)
chromosome3
file: (6135, 36)
df_qtl: (3646, 36)
df_hf: (3437, 36)
df_nel: (2521, 36)
ins_df: (1380, 37)
ins_dff: (609, 8)
result_dff: (164, 10)
chromosome4
file: (8728, 36)
df_qtl: (6689, 36)
df_hf: (6494, 36)
df_nel: (5466, 36)
ins_df: (2837, 37)
ins_dff: (1038, 8)
result_dff: (159, 10)
chromosome5
file: (5981, 36)
df_qtl: (4305, 36)
df_hf: (4096, 36)
df_nel: (3650, 36)
ins_df: (1757, 37)
ins_dff: (597, 8)
result_dff: (175, 10)
chromosome6
file: (10566, 36)
df_qtl: (6623, 36)
df_hf: (6089, 36)
df_nel: (5339, 36)
ins_df: (2087, 37)
ins_dff: (647, 8)
result_dff: (164, 10)
chromosome7
file: (6838, 36)
df_qtl: (3637, 36)
df_hf: (3441, 36)
df_nel: (2636, 36)
ins_df: (1415, 37)
ins_dff: (549, 8)
result_dff: 

In [ ]:
# Table Preparation
f1=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/1/inslov_count1.csv")
f2=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/2/inslov_count2.csv")
f3=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/3/inslov_count3.csv")
f4=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/4/inslov_count4.csv")
f5=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/5/inslov_count5.csv")
f6=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/6/inslov_count6.csv")
f7=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/7/inslov_count7.csv")
f8=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/8/inslov_count8.csv")
f9=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/9/inslov_count9.csv")
f10=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/10/inslov_count10.csv")
f11=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/11/inslov_count11.csv")
f12=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/12/inslov_count12.csv")
f13=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/13/inslov_count13.csv")
f14=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/14/inslov_count14.csv")
f15=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/15/inslov_count15.csv")
f16=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/16/inslov_count16.csv")
f17=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/17/inslov_count17.csv")
f18=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/18/inslov_count18.csv")
f19=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/19/inslov_count19.csv")
f20=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/20/inslov_count20.csv")
f21=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/21/inslov_count21.csv")
f22=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/22/inslov_count22.csv")
f23=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/23/inslov_count23.csv")
f24=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/24/inslov_count24.csv")
f25=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/25/inslov_count25.csv")
f26=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/26/inslov_count26.csv")
f27=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/27/inslov_count27.csv")
f28=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/28/inslov_count28.csv")
f29=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/29/inslov_count29.csv")
fX=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/ins/X/inslov_countX.csv")

In [ ]:
table_insqtl=pd.DataFrame(columns=["Chromosome number"]+list(f1))
table_insqtl

,Chromosome number,Unnamed: 0,Name,Symbol,NelLocus,NelProt,QTL Class,ins_count,ins_count_LOV10,Length_LOV10,ins_count_LOV50,Length_LOV50


In [ ]:
f1["Chromosome number"]=1
f2["Chromosome number"]=2
f3["Chromosome number"]=3
f4["Chromosome number"]=4
f5["Chromosome number"]=5
f6["Chromosome number"]=6
f7["Chromosome number"]=7
f8["Chromosome number"]=8
f9["Chromosome number"]=9
f10["Chromosome number"]=10
f11["Chromosome number"]=11
f12["Chromosome number"]=12
f13["Chromosome number"]=13
f14["Chromosome number"]=14
f15["Chromosome number"]=15
f16["Chromosome number"]=16
f17["Chromosome number"]=17
f18["Chromosome number"]=18
f19["Chromosome number"]=19
f20["Chromosome number"]=20
f21["Chromosome number"]=21
f22["Chromosome number"]=22
f23["Chromosome number"]=23
f24["Chromosome number"]=24
f25["Chromosome number"]=25
f26["Chromosome number"]=26
f27["Chromosome number"]=27
f28["Chromosome number"]=28
f29["Chromosome number"]=29
fX["Chromosome number"]='X'

In [ ]:
table_insqtl=pd.concat([table_insqtl, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, fX])

In [ ]:
table_insqtl.shape

(3609, 12)

In [ ]:
table_insqtl.head

<bound method NDFrame.head of    Chromosome number Unnamed: 0  \
0                  1          0   
1                  1          1   
2                  1          2   
3                  1          3   
4                  1          4   
..               ...        ...   
54                 X         54   
55                 X         55   
56                 X         56   
57                 X         57   
58                 X         58   

                                             Name        Symbol NelLocus  \
0             1,4-alpha-glucan branching enzyme 1          GBE1     GBE1   
1                 3-hydroxyacyl-CoA dehydratase 2         HACD2    HACD2   
2             ABI family member 3 binding protein        ABI3BP   ABI3BP   
3       ATP binding cassette subfamily G member 1         ABCG1    ABCG1   
4                                     ATPase 13A3       ATP13A3  ATP13A3   
..                                            ...           ...      ...   
54               

In [ ]:
table_insqtl.to_csv("table_insqtl.csv")

shutil.copy("table_insqtl.csv", "/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal")

'/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/table_insqtl.csv'

In [ ]:
# deletion analysis

import shutil
import pandas as pd

chr=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,"X"]
for i in chr:
  in_f1=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/{i}/hfqtl_NelVar{i}.csv"
  op_loc=f"/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/{i}"
  print(f"chromosome{i}")

  df=pd.read_csv(in_f1)
  print("file:", df.shape)

  df_qtl=df[df['qtlgene_type']=="within_gene"]
  print("df_qtl:", df_qtl.shape)

  df_hf=df_qtl[df_qtl["HFgene_type"]=="within_gene"]
  print("df_hf:", df_hf.shape)

  df_nel=df_hf[df_hf["Nelgene_type"]=="within_gene"]
  print("df_nel:", df_nel.shape)

  del_df=df_nel.drop_duplicates(subset=["Symbol", "QTL Class", "startpos.q", "endpos.q", "LOV"]).reset_index()
  print("del_df:", del_df.shape)

  out_f1=f"HFNelQTL_wd{i}.csv"
  del_df.to_csv(out_f1)
  shutil.copy(out_f1, op_loc)

  del_dff=del_df.groupby(["Name", "Symbol", "NelLocus", "NelProt", "startpos.q", "endpos.q", "LOV"]).agg({"QTL Class": lambda x: ",".join(set(x))}).reset_index()
  print("del_dff:",del_dff.shape)

  out_ff1=f"HFNelQTL{i}.csv"
  del_dff.to_csv(out_ff1)
  shutil.copy(out_ff1, op_loc)


  # Add a count column
  del_dff["del_count"]=del_dff.groupby(["Name", "Symbol", "NelLocus", "NelProt"])["Symbol"].transform("count")

  # Add LOV >= 10 and Length columns
  del_dff["del_count_LOV10"]=(del_dff["LOV"]>=10).astype(int)
  del_dff["Length_LOV10"]=del_dff["LOV"].where(del_dff["LOV"]>=10, None)

  # Add LOV>=50 and its length columns
  del_dff["del_count_LOV50"]=(del_dff["LOV"]>=50).astype(int)
  del_dff["Length_LOV50"]=del_dff["LOV"].where(del_dff["LOV"]>=50, None)

  # Aggregate QTL Class and other columns
  result_dff=del_dff.groupby(["Name", "Symbol", "NelLocus", "NelProt"]).agg({"QTL Class" : lambda x: ",".join(set(x)),
    "del_count": "first", "del_count_LOV10" : 'sum',
    "Length_LOV10" :lambda x: ', '.join(map(str, x.dropna().astype(int))) if not x.dropna().empty else None,
    "del_count_LOV50" : 'sum',
    "Length_LOV50" :lambda x: ', '.join(map(str, x.dropna().astype(int))) if not x.dropna().empty else None }).reset_index()

  out_f2=f"dellov_count{i}.csv"
  result_dff.to_csv(out_f2)
  print("result_dff:", result_dff.shape)
  shutil.copy(out_f2, op_loc)



chromosome1
file: (3433, 36)
df_qtl: (2101, 36)
df_hf: (1965, 36)
df_nel: (1751, 36)
del_df: (942, 37)
del_dff: (320, 8)
result_dff: (105, 10)
chromosome2
file: (2580, 36)
df_qtl: (1548, 36)
df_hf: (1426, 36)
df_nel: (1188, 36)
del_df: (613, 37)
del_dff: (245, 8)
result_dff: (100, 10)
chromosome3
file: (2346, 36)
df_qtl: (1182, 36)
df_hf: (1122, 36)
df_nel: (818, 36)
del_df: (490, 37)
del_dff: (231, 8)
result_dff: (99, 10)
chromosome4
file: (3622, 36)
df_qtl: (2617, 36)
df_hf: (2505, 36)
df_nel: (1885, 36)
del_df: (999, 37)
del_dff: (365, 8)
result_dff: (110, 10)
chromosome5
file: (1728, 36)
df_qtl: (1174, 36)
df_hf: (1071, 36)
df_nel: (915, 36)
del_df: (517, 37)
del_dff: (178, 8)
result_dff: (85, 10)
chromosome6
file: (3873, 36)
df_qtl: (2640, 36)
df_hf: (2453, 36)
df_nel: (2207, 36)
del_df: (823, 37)
del_dff: (260, 8)
result_dff: (95, 10)
chromosome7
file: (3040, 36)
df_qtl: (2007, 36)
df_hf: (1931, 36)
df_nel: (1690, 36)
del_df: (606, 37)
del_dff: (224, 8)
result_dff: (89, 10)
chrom

In [ ]:
# table preparation - deletion

ff1=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/1/dellov_count1.csv")
ff2=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/2/dellov_count2.csv")
ff3=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/3/dellov_count3.csv")
ff4=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/4/dellov_count4.csv")
ff5=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/5/dellov_count5.csv")
ff6=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/6/dellov_count6.csv")
ff7=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/7/dellov_count7.csv")
ff8=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/8/dellov_count8.csv")
ff9=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/9/dellov_count9.csv")
ff10=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/10/dellov_count10.csv")
ff11=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/11/dellov_count11.csv")
ff12=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/12/dellov_count12.csv")
ff13=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/13/dellov_count13.csv")
ff14=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/14/dellov_count14.csv")
ff15=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/15/dellov_count15.csv")
ff16=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/16/dellov_count16.csv")
ff17=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/17/dellov_count17.csv")
ff18=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/18/dellov_count18.csv")
ff19=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/19/dellov_count19.csv")
ff20=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/20/dellov_count20.csv")
ff21=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/21/dellov_count21.csv")
ff22=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/22/dellov_count22.csv")
ff23=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/23/dellov_count23.csv")
ff24=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/24/dellov_count24.csv")
ff25=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/25/dellov_count25.csv")
ff26=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/26/dellov_count26.csv")
ff27=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/27/dellov_count27.csv")
ff28=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/28/dellov_count28.csv")
ff29=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/29/dellov_count29.csv")
ffX=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/del/X/dellov_countX.csv")


In [ ]:
table_delqtl=pd.DataFrame(columns=["Chromosome number"]+list(ff1))
table_delqtl

,Chromosome number,Unnamed: 0,Name,Symbol,NelLocus,NelProt,QTL Class,del_count,del_count_LOV10,Length_LOV10,del_count_LOV50,Length_LOV50


In [ ]:
ff1["Chromosome number"]=1
ff2["Chromosome number"]=2
ff3["Chromosome number"]=3
ff4["Chromosome number"]=4
ff5["Chromosome number"]=5
ff6["Chromosome number"]=6
ff7["Chromosome number"]=7
ff8["Chromosome number"]=8
ff9["Chromosome number"]=9
ff10["Chromosome number"]=10
ff11["Chromosome number"]=11
ff12["Chromosome number"]=12
ff13["Chromosome number"]=13
ff14["Chromosome number"]=14
ff15["Chromosome number"]=15
ff16["Chromosome number"]=16
ff17["Chromosome number"]=17
ff18["Chromosome number"]=18
ff19["Chromosome number"]=19
ff20["Chromosome number"]=20
ff21["Chromosome number"]=21
ff22["Chromosome number"]=22
ff23["Chromosome number"]=23
ff24["Chromosome number"]=24
ff25["Chromosome number"]=25
ff26["Chromosome number"]=26
ff27["Chromosome number"]=27
ff28["Chromosome number"]=28
ff29["Chromosome number"]=29
ffX["Chromosome number"]='X'

In [ ]:
table_delqtl=pd.concat([table_delqtl, ff1, ff2, ff3, ff4, ff5, ff6, ff7, ff8, ff9, ff10, ff11, ff12, ff13, ff14, ff15, ff16, ff17, ff18, ff19, ff20, ff21, ff22, ff23, ff24, ff25, ff26, ff27, ff28, ff29, ffX])

In [ ]:
table_delqtl.shape

(2005, 12)

In [ ]:
table_delqtl.to_csv("table_delqtl.csv")

shutil.copy("table_delqtl.csv", "/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal")

'/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/table_delqtl.csv'

In [ ]:
# substitution analysis
sub_df=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/subs/1/hfqtl_NelVar1.csv")

In [ ]:
sub_df.shape

(296139, 36)

In [ ]:
sub_qtl=sub_df[sub_df["qtlgene_type"]=="within_gene"]
sub_qtl.shape

(211963, 36)

In [ ]:
sub_hf=sub_qtl[sub_qtl["HFgene_type"]=="within_gene"]
sub_hf.shape

(210381, 36)

In [ ]:
sub_nel=sub_hf[sub_hf["Nelgene_type"]=="within_gene"]
sub_nel.shape

(208705, 36)

In [ ]:
sub_df=sub_nel.drop_duplicates(subset=["Symbol", "QTL Class", "startpos.q", "endpos.q"]).reset_index()
sub_df.shape

(102305, 37)

In [ ]:
sub_df=sub_df.groupby(["Name", "Symbol", "NelLocus", "NelProt", "startpos.q", "endpos.q"]).agg({"QTL Class": lambda x: ",".join(set(x))}).reset_index()
sub_df.shape

(36264, 7)

In [ ]:
sub_df.to_csv("sub_df.csv")

In [ ]:
sub_df["sub_count"]=sub_df.groupby(["Name", "Symbol", "NelLocus", "NelProt"])["Symbol"].transform("count")


# Aggregate QTL Class and other columns
result_dff=sub_df.groupby(["Name", "Symbol", "NelLocus", "NelProt"]).agg({"QTL Class" : lambda x: ",".join(set(x)),
    "sub_count": "first"}).reset_index()




In [ ]:
result_dff.shape

(298, 6)

In [ ]:
result_dff.to_csv("result_dff.csv")

In [ ]:
import shutil
import pandas as pd

chr=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,"X"]
for i in chr:
  in_f1=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/subs/{i}/hfqtl_NelVar{i}.csv"
  op_loc=f"/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/{i}"
  print(f"chromosome{i}")

  df=pd.read_csv(in_f1)
  print("file:", df.shape)

  df_qtl=df[df['qtlgene_type']=="within_gene"]
  print("df_qtl:", df_qtl.shape)

  df_hf=df_qtl[df_qtl["HFgene_type"]=="within_gene"]
  print("df_hf:", df_hf.shape)

  df_nel=df_hf[df_hf["Nelgene_type"]=="within_gene"]
  print("df_nel:", df_nel.shape)

  subs_df=df_nel.drop_duplicates(subset=["Symbol", "QTL Class", "startpos.q", "endpos.q"]).reset_index()
  print("subs_df:", subs_df.shape)

  out_f1=f"HFNelQTL_wd{i}.csv"
  sub_df.to_csv(out_f1)
  shutil.copy(out_f1, op_loc)

  subs_dff=subs_df.groupby(["Name", "Symbol", "NelLocus", "NelProt", "startpos.q", "endpos.q"]).agg({"QTL Class": lambda x: ",".join(set(x))}).reset_index()
  print("subs_dff:",subs_dff.shape)

  out_ff1=f"HFNelQTL{i}.csv"
  subs_dff.to_csv(out_ff1)
  shutil.copy(out_ff1, op_loc)


  # Add a count column
  subs_dff["subs_count"]=subs_dff.groupby(["Name", "Symbol", "NelLocus", "NelProt"])["Symbol"].transform("count")

  # Aggregate QTL Class and other columns
  result_dfff=subs_dff.groupby(["Name", "Symbol", "NelLocus", "NelProt"]).agg({"QTL Class" : lambda x: ",".join(set(x)),
    "subs_count": "first"}).reset_index()

  out_f2=f"subs_count{i}.csv"
  result_dfff.to_csv(out_f2)
  print("result_dfff:", result_dfff.shape)
  shutil.copy(out_f2, op_loc)

  subs50_count=result_dfff[result_dfff["subs_count"]>=50]
  out_f3=f"subs50_count{i}.csv"
  subs50_count.to_csv(out_f3)
  print("subs50_count:", subs50_count.shape)
  shutil.copy(out_f3, op_loc)



chromosome1
file: (296139, 36)
df_qtl: (211963, 36)
df_hf: (210381, 36)
df_nel: (208705, 36)
subs_df: (102305, 37)
subs_dff: (36264, 7)
result_dfff: (298, 6)
subs50_count: (139, 6)
chromosome2
file: (261762, 36)
df_qtl: (189829, 36)
df_hf: (185979, 36)
df_nel: (171815, 36)
subs_df: (105502, 37)
subs_dff: (40332, 7)
result_dfff: (346, 6)
subs50_count: (168, 6)
chromosome3
file: (242803, 36)
df_qtl: (168115, 36)
df_hf: (166740, 36)
df_nel: (160586, 36)
subs_df: (82766, 37)
subs_dff: (35304, 7)
result_dfff: (412, 6)
subs50_count: (161, 6)
chromosome4
file: (385030, 36)
df_qtl: (332070, 36)
df_hf: (330758, 36)
df_nel: (325214, 36)
subs_df: (149747, 37)
subs_dff: (56558, 7)
result_dfff: (257, 6)
subs50_count: (156, 6)
chromosome5
file: (341901, 36)
df_qtl: (291457, 36)
df_hf: (289872, 36)
df_nel: (285744, 36)
subs_df: (132037, 37)
subs_dff: (45865, 7)
result_dfff: (473, 6)
subs50_count: (202, 6)
chromosome6


<ipython-input-17-1dc510863e94>:10: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(in_f1)


file: (377023, 36)
df_qtl: (322460, 36)
df_hf: (317637, 36)
df_nel: (313766, 36)
subs_df: (136827, 37)
subs_dff: (38630, 7)
result_dfff: (324, 6)
subs50_count: (138, 6)
chromosome7
file: (281111, 36)
df_qtl: (204548, 36)
df_hf: (203939, 36)
df_nel: (183777, 36)
subs_df: (96415, 37)
subs_dff: (36260, 7)
result_dfff: (399, 6)
subs50_count: (157, 6)
chromosome8
file: (313709, 36)
df_qtl: (267707, 36)
df_hf: (266219, 36)
df_nel: (264136, 36)
subs_df: (100111, 37)
subs_dff: (36731, 7)
result_dfff: (265, 6)
subs50_count: (144, 6)
chromosome9
file: (165723, 36)
df_qtl: (121886, 36)
df_hf: (120815, 36)
df_nel: (118868, 36)
subs_df: (57111, 37)
subs_dff: (23143, 7)
result_dfff: (192, 6)
subs50_count: (104, 6)
chromosome10
file: (734603, 36)
df_qtl: (668328, 36)
df_hf: (665570, 36)
df_nel: (660485, 36)
subs_df: (121408, 37)
subs_dff: (38575, 7)
result_dfff: (303, 6)
subs50_count: (146, 6)
chromosome11
file: (316060, 36)
df_qtl: (255889, 36)
df_hf: (253982, 36)
df_nel: (252177, 36)
subs_df: (1159

<ipython-input-17-1dc510863e94>:10: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(in_f1)


file: (99945, 36)
df_qtl: (59028, 36)
df_hf: (58819, 36)
df_nel: (51655, 36)
subs_df: (28112, 37)
subs_dff: (10461, 7)
result_dfff: (84, 6)
subs50_count: (43, 6)
chromosome28
file: (133490, 36)
df_qtl: (100418, 36)
df_hf: (99687, 36)
df_nel: (91590, 36)
subs_df: (60741, 37)
subs_dff: (21258, 7)
result_dfff: (116, 6)
subs50_count: (71, 6)
chromosome29
file: (172161, 36)
df_qtl: (138524, 36)
df_hf: (138371, 36)
df_nel: (137642, 36)
subs_df: (67779, 37)
subs_dff: (21004, 7)
result_dfff: (242, 6)
subs50_count: (77, 6)
chromosomeX
file: (118027, 36)
df_qtl: (60820, 36)
df_hf: (59453, 36)
df_nel: (56371, 36)
subs_df: (29836, 37)
subs_dff: (12197, 7)
result_dfff: (194, 6)
subs50_count: (70, 6)


In [ ]:
# table preparation - substitution

fff1=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/1/subs50_count1.csv")
fff2=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/2/subs50_count2.csv")
fff3=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/3/subs50_count3.csv")
fff4=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/4/subs50_count4.csv")
fff5=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/5/subs50_count5.csv")
fff6=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/6/subs50_count6.csv")
fff7=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/7/subs50_count7.csv")
fff8=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/8/subs50_count8.csv")
fff9=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/9/subs50_count9.csv")
fff10=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/10/subs50_count10.csv")
fff11=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/11/subs50_count11.csv")
fff12=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/12/subs50_count12.csv")
fff13=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/13/subs50_count13.csv")
fff14=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/14/subs50_count14.csv")
fff15=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/15/subs50_count15.csv")
fff16=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/16/subs50_count16.csv")
fff17=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/17/subs50_count17.csv")
fff18=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/18/subs50_count18.csv")
fff19=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/19/subs50_count19.csv")
fff20=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/20/subs50_count20.csv")
fff21=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/21/subs50_count21.csv")
fff22=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/22/subs50_count22.csv")
fff23=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/23/subs50_count23.csv")
fff24=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/24/subs50_count24.csv")
fff25=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/25/subs50_count25.csv")
fff26=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/26/subs50_count26.csv")
fff27=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/27/subs50_count27.csv")
fff28=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/28/subs50_count28.csv")
fff29=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/29/subs50_count29.csv")
fffX=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/sub/X/subs50_countX.csv")


In [ ]:
table_subsqtl50=pd.DataFrame(columns=["Chromosome number"]+list(fff1))
table_subsqtl50

,Chromosome number,Unnamed: 0,Name,Symbol,NelLocus,NelProt,QTL Class,subs_count


In [ ]:
fff1["Chromosome number"]=1
fff2["Chromosome number"]=2
fff3["Chromosome number"]=3
fff4["Chromosome number"]=4
fff5["Chromosome number"]=5
fff6["Chromosome number"]=6
fff7["Chromosome number"]=7
fff8["Chromosome number"]=8
fff9["Chromosome number"]=9
fff10["Chromosome number"]=10
fff11["Chromosome number"]=11
fff12["Chromosome number"]=12
fff13["Chromosome number"]=13
fff14["Chromosome number"]=14
fff15["Chromosome number"]=15
fff16["Chromosome number"]=16
fff17["Chromosome number"]=17
fff18["Chromosome number"]=18
fff19["Chromosome number"]=19
fff20["Chromosome number"]=20
fff21["Chromosome number"]=21
fff22["Chromosome number"]=22
fff23["Chromosome number"]=23
fff24["Chromosome number"]=24
fff25["Chromosome number"]=25
fff26["Chromosome number"]=26
fff27["Chromosome number"]=27
fff28["Chromosome number"]=28
fff29["Chromosome number"]=29
fffX["Chromosome number"]='X'

In [ ]:
table_subsqtl50=pd.concat([table_subsqtl50, fff1, fff2, fff3, fff4, fff5, fff6, fff7, fff8, fff9, fff10, fff11, fff12, fff13, fff14, fff15, fff16, fff17, fff18, fff19, fff20, fff21, fff22, fff23, fff24, fff25, fff26, fff27, fff28, fff29, fffX])

In [ ]:
table_subsqtl50.shape

(3411, 8)

In [ ]:
table_subsqtl50.to_csv("table_subsqtl50.csv")

shutil.copy("table_subsqtl50.csv", "/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal")

'/content/drive/MyDrive/Hereford_VarAnalysis/HFNelQTL_VarAnal/table_subsqtl50.csv'

In [ ]:
import pandas as pd

In [ ]:
sdf=pd.read_csv("/content/table_subsqtl50 (1).csv")

In [ ]:
sdf.shape

(3411, 7)

In [ ]:
sdf_wd=sdf.drop_duplicates(subset=["Symbol", "NelLocus"])

In [ ]:
sdf_wd.shape

(3410, 7)

In [ ]:
sdf_wd1=sdf.drop_duplicates(subset=["Symbol"])

In [ ]:
sdf_wd1.shape

(3318, 7)

In [ ]:
sdf_wd2=sdf.drop_duplicates(subset=["NelLocus"])

In [ ]:
sdf_wd2.shape

(3308, 7)